## 在划分数据集之前之后信息发生的变化称为信息增益，获得信息增益最高的特征就是最好的选择（ID3法）。
- 信息的期望称之为熵。假设一组数值/字符串（xt）分为n类，在第i类中，xt的信息指的是落在i类的概率的对数的负值l(xit)=-log2(p(xit))，则该组的熵为ΣtΣi[p(xit)l(xit)]
- 熵越大，混合的数据也越多，无序程度越大。通过不同的划分使组的熵最小（信息增益划分法）
- 另一种度量集合无序程度的方法是基尼不纯度（Gini impurity），简单地说就是从一个数据集中随机选取子项，度量其被错误分类到其他分组里的概率。

In [39]:
#..........................案例1:
#.......................... 理解信息增益
from math import log
import operator
# 数据集
dataSet = [[1, 1, 'yes'],[1, 1, 'yes'],[1, 0, 'no'],[0, 1, 'no'],[0, 1, 'no']]
# 数据集的熵（信息的期望值）
# 若不分组，即1组，计算yes和no的熵，yes信息和no信息的求和
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) #log base 2
    return shannonEnt
print("dataSet最后一列的熵值：",calcShannonEnt(dataSet))
dataSet[0][-1]='maybe'
print("dataSet增加一种分类后最后一列的熵值：",calcShannonEnt(dataSet))
# 抽取数据
def splitDataSet(dataSet, axis, value): # axis列表中的第几个，value根据值筛选
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]  # 删掉取值的这一类
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet
# 若分组，即根据前面的信息分组，计算yes和no的熵，yes信息和no信息的组内和组间的总和
# 选择最好的划分方式
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1     # 最后一列是标签
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures):        #iterate over all the features
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)       # 每一列的分类标签列表
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value) # 用特征划分数据集
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet) # 求每个数据集的熵并求和
        infoGain = baseEntropy - newEntropy     # 信息增益
        if (infoGain > bestInfoGain):       #compare this to the best gain so far
            bestInfoGain = infoGain         # 挑选最好的信息增益
            bestFeature = i
    return bestFeature
print("dataSet的最好的分类列：第%s列" % chooseBestFeatureToSplit(dataSet))

dataSet最后一列的熵值： 0.9709505944546686
dataSet增加一种分类后最后一列的熵值： 1.3709505944546687
dataSet的最好的分类列：第0列


In [40]:
#..........................案例2:
#.......................... 理解决策树
# 设定输出
label = ['no surfacing','flippers']
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]
# 决策树
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): # 如果分类变量是常数，不再分类
        return classList[0]
    if len(dataSet[0]) == 1: # 遍历完所有特征
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet) # 选出最好的一列
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       #copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree
print("dataSet的决策树",createTree(dataSet,label))
# 决策树的可视化，这里先跳过
# 决策树估计
def classify(inputTree,featLabels,testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict): # 是否仍是字典
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else: classLabel = valueOfFeat
    return classLabel
label = ['no surfacing','flippers'] # createTree会改变label的值，因此每次都需要重新定义
mytree=createTree(dataSet,label)
label = ['no surfacing','flippers']
print("预测结果：",classify(mytree,label,[1,1]))
print("预测结果：",classify(mytree,label,[0,1]))


dataSet的决策树 {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'maybe'}}}}
预测结果： maybe
预测结果： no
